# Estimación probabilística de la dosis de exposición dérmica


## Estimación de la concentración de la sustancia de interés en la solución a considerar

- *mean* corresponde al valor de la media de la concentración de la sustancia de interés en la solución a considerar.
- *stdev* es la desviación estándar de la concentración de la sustancia de interés en la solución utilizada.
- *CV* es el coeficiente de variación es la desviación estándar relativa  de la concentración de la sustancia de interés en la solución utilizada.
- *dosis* es la cantidad de mg de ingrediente activo por hectarea (mg *i.a*/Ha) a ser utilizada durante la aplicación del producto químico de uso agrícola de que se trate.
- *volumen de aspersión* es el volumen por hectarea (L/Ha) de solución o dispersión de la sustancia de interés a ser utilizada.


In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from scipy.stats import lognorm
from scipy.stats import iqr
import ipywidgets as widgets
from ipywidgets import Checkbox
from ipywidgets import interactive_output
from IPython.display import display
from IPython.display import clear_output
%matplotlib inline

dtcnc = []

# Define the function to be plotted
def plot_function(dose_per_ha, volasp, cv):
    global dtcnc
    mean = dose_per_ha / volasp
    stdev = cv * mean
    
    phi = (stdev ** 2 + mean ** 2) ** 0.5
    mu = np.log(mean ** 2 / phi)
    sigma = (np.log(phi ** 2 / mean ** 2)) ** 0.5
    
    dtcnc = np.random.lognormal(mu, sigma, 5000)
    
    plt.figure(figsize=(10, 8))
    plt.hist(dtcnc, bins=30, density=True, alpha=0.5, color="b")
    
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 1000)
    p = lognorm.pdf(x=x, scale=mean, s=sigma)
    plt.plot(x, p, 'k', linewidth=2)
    
    plt.ylabel('Densidad de probabilidad')
    plt.xlabel('mg/cm$^{3}$', y=0.05)
    
    plt.savefig('log_normal_sln.png', bbox_inches='tight', dpi=300)
    plt.show()

# Define the slider widgets
slider_dose_per_ha = widgets.FloatSlider(min=0, max=1000, step=1, value=100, description='mg i.a/Ha')
slider_volasp = widgets.FloatSlider(min=0, max=2000, step=0.1, value=100, description='L/hectarea')
slider_cv = widgets.FloatSlider(min=0, max=2, step=0.01, value=0.1, description='CV')

# Create the interactive plot
interactive_plot = widgets.interactive(plot_function, dose_per_ha=slider_dose_per_ha, volasp=slider_volasp, cv=slider_cv)
display(interactive_plot)

dtcnc = interactive_plot.result


interactive(children=(FloatSlider(value=100.0, description='mg i.a/Ha', max=1000.0, step=1.0), FloatSlider(val…

# Estimación del coeficiente de permeabilidad dérmica

*Ecuación extraida del Risk assessment guidance for superfund volume I: Human health evaluation manual (Part E, supplemental guidance for dermal risk assessment. A.1.1 Estimation of Kp for organic compounds)*

$log(Kp) = -2.80+(0.66*log(Kw)-(0.0056*mw)$


$C_{permeabilidad} = 10^{log(Kp)}$

- *lgkw* es el logaritmo del coeficiente de partición octanol-agua.
- *mw* hace referencia al peso molecular de la sustancia de interés, en g/mol.

In [2]:
def lgkp(lgkw, mw):
    logKp = -2.80+(0.66*lgkw)-(0.0056*mw)
    prmcf = 10**(logKp)
    return prmcf

slider_lgkw1 = widgets.FloatSlider(description='lgkw', min=-10, step=0.1, value=3.4)
slider_mw1 = widgets.FloatSlider(description='mw', min=0, max=10000, step=0.1, value=650.7)
out = widgets.Output()

def update(change):
    with out:
        out.clear_output()
        prmcf = lgkp(slider_lgkw1.value, slider_mw1.value)
        print("Coeficiente de permeabilidad dérmica de la sustancia:", prmcf)

slider_lgkw1.observe(update, 'value')
slider_mw1.observe(update, 'value')

display(widgets.VBox([slider_lgkw1, slider_mw1, out]))

In [3]:
areakde = pd.read_csv('areakde.csv').loc[:, '0'].values

In [4]:
pesokde = pd.read_csv('pesokde.csv').loc[:, '0'].values

## Resultados de la estimación probabilística de las dosis de exposición mediante el uso del método de Monte Carlo

*Distribución de la concentración de la sustancia de interés en la solución a considerar; distribución estadística de las dosis de exposición y diagrama de cajas (boxplot) de esas dosis de exposición. Resaltado en rojo se encuentra el percentil 97.5 de la dosis de exposición*.

Se puede obtener un intervalo de las dosis de exposición dérmica que corresponde al uso de elementos de protección personal (EPP), tales como coverall, coverall certificado, guantes y máscara facial certificados, así como a cualquier combinación de su uso. También, existe la posibilidad de obtener el intervalo de las dosis de exposición dérmica para el caso de no utilizar ningún elemento de protección personal; en ese caso, se asume que el porcentaje de área corporal expuesta es de 33 %.
- *prcrdrmxpt* es el porcentaje de área dérmica expuesta.
- *prcbsrdrm* corresponde al porcentaje de absorción dérmica del total de la dosis dérmica disponible.
- *tm* hace referencia al tiempo de exposición, en horas.
- *bins* es el número de intervalos empleados para obtener el histograma de la distribución de las dosis de exposición.

In [5]:
def hstgrm(xpdrm, bins, xlim):
    plt.figure(figsize=(10,8))
    plt.xlim(xlim)
    plt.hist(xpdrm, density=True, bins=bins)  
    plt.ylabel('Densidad de probabilidad')
    plt.xlabel('mg/kg·día');
    plt.axvline(x = np.percentile(xpdrm, 50), linewidth=1, color='r', linestyle=':')
    plt.axvline(x = np.percentile(xpdrm, 2.5), linewidth=1, color='r', linestyle='--')
    plt.axvline(x = np.percentile(xpdrm, 97.5), linewidth=1, color='r', linestyle='dashed')
    plt.text((np.percentile(xpdrm, 97.5)+0.00005*iqr(xpdrm)), len(xpdrm)/sum(xpdrm), 'P97.5 = '+np.format_float_scientific(np.percentile(xpdrm, 97.5), precision=2), rotation=90, color='red', verticalalignment='center')
    plt.savefig('exporal_dermica.png', bbox_inches='tight', dpi=300)
    plt.show()
    
def montecarlo(prcrdrmxpt, prcbsrdrm, dtcnc, lgkw, mw, tm, N, coverall, coverallcert, guantes, masc, mandwn, manupp, vehdwn, vehupp):
    global prmcf
    random_concsolap = np.random.choice(dtcnc, size=N)
    random_ardrmc = np.random.choice(areakde, size=N)
    random_peso = np.random.choice(pesokde, size=N)
    prmcf = lgkp(lgkw, mw)
    
    multiplier = 1
    if mandwn or manupp:
        multiplier *= 4
    if vehdwn:
        multiplier *= 10
    if vehupp:
        multiplier *= 8
        
    if coverall and coverallcert:
        expderm = (((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm))/random_peso)*multiplier
    if coverall:
        if guantes == False:
            if masc == False:
                cvrllrdccn = 0.1
                expderm = (((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm)*cvrllrdccn)/random_peso)*multiplier
            else:
                cvrllrdccn = 0.1
                masc=0.8
                expderm = ((((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm)*cvrllrdccn)*masc)/random_peso)*multiplier
        else:
            cvrllrdccn = 0.1
            guantes = 0.1
            masc=0.8
            expderm = (((((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm)*cvrllrdccn)*guantes)*masc)/random_peso)*multiplier
    elif coverallcert:
        if guantes == False:
            if masc == False:
                cvrllrdccn = 0.05
                expderm = (((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm)*cvrllrdccn)/random_peso)*multiplier
            else:
                cvrllrdccn = 0.05
                masc = 0.8
                expderm = ((((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm)*cvrllrdccn)*masc)/random_peso)*multiplier
        else:
            cvrllrdccn = 0.05
            guantes = 0.1
            masc:0.8
            expderm = (((((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm)*cvrllrdccn)*guantes)*masc)/random_peso)*multiplier
    elif guantes:
        if masc == False:
            guantes = 0.9
            expderm = (((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm)*guantes)/random_peso)*multiplier
        else:
            guantes = 0.9
            masc=0.8
            expderm = ((((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm)*guantes)*masc)/random_peso)*multiplier
    elif masc:
        if guantes == False:
            masc = 0.8
            expderm = (((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm)*masc)/random_peso)*multiplier
        else:
            expderm = ((((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm)*guantes)*masc)/random_peso)*multiplier
    else:
        expderm = (((random_concsolap*random_ardrmc*prcrdrmxpt)*(prcbsrdrm*prmcf*tm))/random_peso)*multiplier
    return expderm

def hstgrm_wrapper(prcrdrmxpt, prcbsrdrm, tm, bins, coverall, coverallcert, guantes, masc, mandwn, manupp, vehdwn, vehupp):
    global dtcnc, prmcf
    xpdrm = montecarlo(prcrdrmxpt, prcbsrdrm, dtcnc, slider_lgkw.value, slider_mw.value, tm, 10000, coverall, coverallcert, guantes, masc, mandwn, manupp, vehdwn, vehupp)
    xlim = (0, np.percentile(xpdrm, 97.5)+iqr(xpdrm)+(0.01*iqr(xpdrm)))
    plt.clf()
    plt.figure(figsize=(10,8))
    hstgrm(xpdrm, bins, xlim)
    plt.figure(figsize=(10,8))
    plt.boxplot(np.array(xpdrm)[np.array(xpdrm < np.percentile(xpdrm, 97.5)+iqr(xpdrm))], vert=False, patch_artist=True)
    plt.xlabel('mg/kg·día')
    plt.yticks([1], [''])
    plt.axvline(x = np.quantile(xpdrm, 0.975) , linewidth=1, color='r', linestyle='--')
    plt.text((np.percentile(xpdrm, 97.5)+(np.percentile(xpdrm, 97.5)*1.0e-05)), 1.125, 'P97.5 = '+np.format_float_scientific(np.percentile(xpdrm, 97.5), precision=2), color='red', rotation=90)
    plt.xlim(xlim)
    plt.savefig('boxplot_dermica.png', bbox_inches='tight', dpi=300)
    plt.show()
    
checkbox_cvrll = Checkbox(value=False, description='uso de coverall')
checkbox_cvrllcrtf = Checkbox(value=False, description='uso de coverall certificado')

checkbox_mandwn = Checkbox(value=False, description='aplicación manual abajo')
checkbox_manupp = Checkbox(value=False, description='aplicación manual arriba')
checkbox_vehdwn = Checkbox(value=False, description='aplicación con vehículo abajo')
checkbox_vehupp = Checkbox(value=False, description='aplicación con vehículo arriba')

def on_checkbox_cvrll_change(change):
    if change['new'] == True:
        checkbox_cvrllcrtf.value = False
        
def on_checkbox_cvrllcrtf_change(change):
    if change['new'] == True:
        checkbox_cvrll.value = False
        
def on_checkbox_mandwn_change(change):
    if change['new'] == True:
        checkbox_manupp.value = False
        checkbox_vehdwn.value = False
        checkbox_vehupp.value = False

def on_checkbox_manupp_change(change):
    if change['new'] == True:
        checkbox_mandwn.value = False
        checkbox_vehdwn.value = False
        checkbox_vehupp.value = False

def on_checkbox_vehdwn_change(change):
    if change['new'] == True:
        checkbox_manupp.value = False
        checkbox_mandwn.value = False
        checkbox_vehupp.value = False

def on_checkbox_vehupp_change(change):
    if change['new'] == True:
        checkbox_manupp.value = False
        checkbox_mandwn.value = False
        checkbox_vehdwn.value = False

checkbox_cvrll.observe(on_checkbox_cvrll_change, names='value')
checkbox_cvrllcrtf.observe(on_checkbox_cvrllcrtf_change, names='value')
checkbox_mandwn.observe(on_checkbox_mandwn_change, names='value')
checkbox_manupp.observe(on_checkbox_manupp_change, names='value')
checkbox_vehdwn.observe(on_checkbox_vehdwn_change, names='value')
checkbox_vehupp.observe(on_checkbox_vehupp_change, names='value')


checkbox_guantes = Checkbox(value=False, description='uso de guantes')
checkbox_masc = Checkbox(value=False, description='uso de máscara facial')
slider_mean = widgets.FloatSlider(min=0, max=10, step=0.01, value=0.07)
slider_stdev = widgets.FloatSlider(min=0, max=10, step=0.01, value=0.07*(1.125/1.07))
slider_prcrdrmxpt = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.33)
slider_prcbsrdrm = widgets.FloatText(min=0.000001, max=1.0, step=0.01, value=0.039)
slider_lgkw = widgets.FloatSlider(description='lgkw', min=-10, step=0.1, value=3.4)
slider_mw = widgets.FloatSlider(description='mw', min=0, max=10000, step=0.1, value=650.7)
slider_tm = widgets.FloatSlider(min=0, max=24, value=24)
slider_bins = widgets.IntSlider(min=1, max=10000, step=1, value=1000)

def interactive_plot_func(dose_per_ha, volasp, stdev, prcrdrmxpt, prcbsrdrm, lgkw, mw, tm, bins, coverall, coverallcert, mandwn, manupp, vehdwn, vehupp, guantes, masc):
    plot_function(dose_per_ha, volasp, stdev)
    hstgrm_wrapper(prcrdrmxpt, prcbsrdrm, tm, bins, coverall, coverallcert, mandwn, manupp, vehdwn, vehupp, guantes, masc)

    
interactive_plot = widgets.interactive(interactive_plot_func,
                                       dose_per_ha=slider_dose_per_ha,
                                       volasp=slider_volasp,
                                       stdev=slider_stdev,
                                       prcrdrmxpt=slider_prcrdrmxpt,
                                       prcbsrdrm=slider_prcbsrdrm,
                                       lgkw=slider_lgkw,
                                       mw=slider_mw,
                                       tm=slider_tm,
                                       bins=slider_bins,
                                       coverall=checkbox_cvrll,
                                       coverallcert=checkbox_cvrllcrtf,
                                       mandwn=checkbox_mandwn,
                                       manupp=checkbox_manupp,
                                       vehdwn=checkbox_vehdwn,
                                       vehupp=checkbox_vehupp,
                                       guantes=checkbox_guantes,
                                       masc=checkbox_masc)
display(interactive_plot)

interactive(children=(FloatSlider(value=100.0, description='mg i.a/Ha', max=1000.0, step=1.0), FloatSlider(val…